## 课程基本信息

- **课程名称**：大数据与商务智能 - 检索增强生成（RAG）系统实践
- **授课对象**：中山大学工商管理非全专硕MBA学生
- **课时**：3学时（上机课）
- **前置知识**：基础Python编程能力，LLM基础应用能力

完成本课程后，您将能够：
1. 独立搭建基于Milvus和LlamaIndex的RAG系统，实现企业知识库检索
2. 掌握向量数据库的索引优化和性能调优方法，提升检索效率30%以上
3. 设计并实现混合检索（密集+稀疏向量）解决方案，提高查询准确率
4. 运用Cherry Studio可视化工具构建完整的企业知识库应用
5. 具备RAG系统部署和维护的基本能力，识别并解决常见问题

## 2.1 Chroma部署

- 纯Python安装（`pip install`即可），无需Docker/部署，1分钟装完
- 能清晰展示「文档向量化→向量入库→相似检索→LLM生成」全流程
- 完全本地运行，无需联网（适合课堂演示）

### 2.1.1 Chroma 安装

可能用时较长，请耐心等待

In [ ]:
!pip install chromadb python-dotenv langchain langchain-community sentence-transformers openai

### 2.1.2 Chroma 相关LLM组件

首先检查Ollama中的本地模型：

In [ ]:
!ollama list

我们需要使用嵌入模型和生成模型，分别在2.1.2.1和2.1.2.2节中进行配置

#### 2.1.2.1 配置Ollama嵌入模型

使用本地知识库需要嵌入模型`bge-m3`，请执行下方命令（用时较短）：

In [ ]:
!ollama pull bge-m3:latest

下面验证嵌入模型是否可用：若下方代码运行后输出前5个向量值，说明模型可用

In [ ]:
# 验证bge-m3模型是否可用
import requests
# 调用Ollama的embeddings接口，测试bge-m3向量化
response = requests.post(
    "http://localhost:11434/api/embeddings",
    json={"model": "bge-m3:latest", "prompt": "MBA财务管理"}
)
print(response.json()["embedding"][:5])  # 若输出前5个向量值，说明模型可用

#### 2.1.2.2 配置Ollama生成模型

为了体现离线部署特性，我们建议使用Ollama中的本地模型，如千问`qwen3`模型，亦可跳过此步，更换为自己喜欢的模型。

运行下方代码块即可安装qwen3模型（可能用时较长，请耐心等待，约15分钟）

In [ ]:
!ollama pull qwen3-vl:8b

### 2.1.3 体验Chroma RAG

#### 2.1.3.1 RAG处理

In [ ]:
import requests
import chromadb
from chromadb.config import Settings
import ollama

class LocalRAG:
    def __init__(self, OLLAMA_API_URL="http://localhost:11434/api", EMBED_MODEL="bge-m3:latest", LLM_MODEL="qwen3-vl:8b", business_cases=None):
        """
        初始化一个完整的本地RAG系统
        params:
            OLLAMA_API_URL: Ollama本地API地址，默认http://localhost:11434/api
            EMBED_MODEL: 本地嵌入模型名称，默认使用bge-m3:latest
            LLM_MODEL: 本地大模型名称，默认使用千问Qwen3-VL:8B
            business_cases: 可选，自定义MBA商业案例列表
        """
        self.OLLAMA_API_URL = OLLAMA_API_URL            # Ollama本地API地址
        self.EMBED_MODEL = EMBED_MODEL                  # 本地嵌入模型，和ollama list中一致
        self.LLM_MODEL = LLM_MODEL                      # 本地大模型名称，和ollama list中一致

        # 初始化Chroma（内存模式）
        print("正在初始化Chroma向量库")
        self.chroma_client = chromadb.Client(Settings(allow_reset=True, anonymized_telemetry=False))
        self.chroma_client.reset()  # 清空旧数据
        self.collection = self.chroma_client.create_collection(
            name="mba_business_cases",
            metadata={"hnsw:space": "cosine"}
        )
        print("Chroma向量库初始化成功")

        # 加载商业案例到Chroma向量库
        print("正在加载MBA商业案例到Chroma向量库")
        self.load_mba_cases(business_cases)
        print("MBA商业案例加载完成")

        print("本地RAG系统初始化完成")

    def load_mba_cases(self, business_cases=None):
        """
        加载MBA商业案例到Chroma向量库
        params:
            business_cases: 可选，自定义MBA商业案例列表
        """
        # 加载MBA商业案例
        business_cases = [
            {"id": "1", "text": "案例1：特斯拉的定价策略——采用撇脂定价法，初期以高价锁定高端用户收回研发成本，后期通过规模化生产降低成本，逐步降价渗透大众市场，最终实现销量和利润双增长；核心逻辑是“高价卡位+低价放量”的组合策略。"},
            {"id": "2", "text": "案例2：星巴克供应链管理——全球统一采购阿拉比卡咖啡豆，在产地附近建立烘焙厂降低物流成本，通过“门店库存动态调整系统”优化补货效率，将缺货率控制在1%以内；核心是“集中采购+本地化加工+数据驱动库存”。"},
            {"id": "3", "text": "案例3：阿里新零售战略——整合淘宝/天猫线上流量和盒马鲜生线下门店，通过“用户行为数据中台”分析消费习惯，实现“线上下单、30分钟送达”的即时零售模式；核心是“线上线下数据打通+履约效率提升”。"},
            {"id": "4", "text": "案例4：华为研发投入——连续10年将营收的15%以上投入研发，聚焦5G通信、芯片设计等核心技术，累计专利数超8万件，构建了难以复制的技术壁垒；核心是“长期主义+核心技术自主可控”。"}
        ] if not business_cases else business_cases

        # 向量化并入库
        texts = [case["text"] for case in business_cases]
        ids = [case["id"] for case in business_cases]
        embeddings = [self.get_embedding(text) for text in texts]
        self.collection.add(documents=texts, embeddings=embeddings, ids=ids)
        print("案例已通过bge-m3向量化，存入Chroma")

    def load_documents(self, nodes):
        """
        Load processed documents (nodes) into Chroma vector database
        params:
            nodes: List of Document nodes from load_and_process_documents
        """
        texts = [node.text for node in nodes]
        ids = [str(i) for i in range(len(nodes))]
        embeddings = [self.get_embedding(text) for text in texts]
        self.collection.add(documents=texts, embeddings=embeddings, ids=ids)
        print(f"Loaded {len(nodes)} document chunks into Chroma")

    # 调用本地嵌入模型bge-m3生成向量
    def get_embedding(self, text):
        """调用本地bge-m3生成文本向量"""
        try:
            response = requests.post(
                f"{self.OLLAMA_API_URL}/embeddings",
                json={"model": self.EMBED_MODEL, "prompt": text},
                timeout=15
            )
            return response.json()["embedding"]
        except Exception as e:
            print(f"bge-m3调用失败：{e}")
            raise e

    # 调用本地生成模型Qwen3-VL:8B生成回答
    def call_local_llm(self, prompt, context):
        """
        调用本地Qwen3-VL:8B生成回答
        :param prompt: 学生的原始提问
        :param context: Chroma检索到的相关MBA案例（上下文信息）
        """
        # 构建RAG提示词
        rag_prompt = f"""
        你是MBA商业课程的专业讲师，请基于以下参考资料回答问题，要求：
        1. 仅使用参考资料中的信息，不编造内容；
        2. 回答简洁、专业，贴合MBA课程的商业视角；
        3. 重点突出案例中的核心策略和商业逻辑。

        参考资料：
        {context}

        学生提问：
        {prompt}
        """

        # 使用Ollama Python库生成回答
        try:
            response = ollama.chat(
            model=self.LLM_MODEL,
            messages=[{"role": "user", "content": rag_prompt}],
            options={
                "temperature": 0.6, 
                "num_ctx": 8192,        # num_ctx=上下文窗口大小
                "stream": True          # 启用流式输出
                } 
            )

            # 提取回答内容
            answer = response["message"]["content"]
            return answer
        except Exception as e:
            print(f"生成模型调用失败：{e}")
            raise e

    def query(self, question, top_k=2):
        """
        RAG问答接口
        params:
            question: 学生提问
            top_k: 检索相关案例数量
        returns:
            answer: 基于检索上下文的回答
        """
        # 向量化学生提问
        question_embedding = self.get_embedding(question)

        # 从Chroma检索相关案例
        results = self.collection.query(
            query_embeddings=[question_embedding],
            n_results=top_k
        )
        retrieved_texts = results['documents'][0]  # 取出文本列表

        print(f"Chroma检索到最相似案例（相似度：{1 - results['distances'][0][0]:.2f}）：\n{retrieved_texts[0]}\n")

        # 构建上下文信息
        context = "\n".join(retrieved_texts)

        # 调用本地LLM生成回答
        print("正在调用本地生成LLM模型回答...")
        answer = self.call_local_llm(question, context)
        return answer

#### 2.1.3.2 实例化一个本地RAG系统

In [ ]:
# 自定义商业案例
rag_business_cases = [
    {"id": "1", "text": "案例1：特斯拉的定价策略——采用撇脂定价法，初期以高价锁定高端用户收回研发成本，后期通过规模化生产降低成本，逐步降价渗透大众市场，最终实现销量和利润双增长；核心逻辑是“高价卡位+低价放量”的组合策略。"},
    {"id": "2", "text": "案例2：星巴克供应链管理——全球统一采购阿拉比卡咖啡豆，在产地附近建立烘焙厂降低物流成本，通过“门店库存动态调整系统”优化补货效率，将缺货率控制在1%以内；核心是“集中采购+本地化加工+数据驱动库存”。"},
    {"id": "3", "text": "案例3：阿里新零售战略——整合淘宝/天猫线上流量和盒马鲜生线下门店，通过“用户行为数据中台”分析消费习惯，实现“线上下单、30分钟送达”的即时零售模式；核心是“线上线下数据打通+履约效率提升”。"},
    {"id": "4", "text": "案例4：华为研发投入——连续10年将营收的15%以上投入研发，聚焦5G通信、芯片设计等核心技术，累计专利数超8万件，构建了难以复制的技术壁垒；核心是“长期主义+核心技术自主可控”。"}
]

# 初始化本地RAG系统
local_rag = LocalRAG(
    OLLAMA_API_URL="http://localhost:11434/api", 
    EMBED_MODEL="bge-m3:latest", 
    LLM_MODEL="qwen3-vl:8b",
    # LLM_MODEL="deepseek-r1:7b",
    business_cases=rag_business_cases
    )

#### 2.1.3.3 体验RAG问答

以下为本地运行的RAG+LLM回答，输出时间可能长达5分钟，期间卡顿是正常情况，请耐心等待回答完成。

In [ ]:
# 提问
query = "哪些企业通过定价策略实现了市场渗透？请分析其核心逻辑。"
print(f"提问：{query}\n")
answer = local_rag.query(query, top_k=2)
print(f"RAG回答：\n{answer}")

## 2.2 RAG优化

### 2.2.1 RAG 提示词工程

#### 2.2.2.1 RAG提示词基本格式


```markdown
# 系统角色
你是企业知识库问答专家，使用提供的检索结果回答问题。

## 检索结果使用规则
- 仅使用检索到的内容回答问题
- 明确引用来源文档和页码
- 对冲突信息标注"信息冲突：来源A认为...来源B认为..."

## 输出格式
**回答**：[基于检索内容的回答]
**来源**：[引用文档列表]
**检索建议**：[如果信息不足，建议补充的检索关键词]
```

#### 2.2.1.2 RAG提示词优化技巧
1. **指定回答风格**：
    ```markdown
    # 回答风格
    - 使用简洁的要点形式回答
    - 每点不超过20个字
    - 重点内容加粗显示
    ```

2. **多轮检索提示**：
    ```markdown
    # 多轮检索优化
    如果第一次检索结果不足以回答问题：
    1. 分析缺失的信息
    2. 生成补充检索关键词
    3. 进行二次检索
    4. 综合两次结果回答
    ```

### 2.2.2 文档加载与预处理

In [ ]:
from llama_index.core import SimpleDirectoryReader, Document
from llama_index.core.node_parser import SentenceSplitter
from typing import List
from datetime import datetime

def load_and_process_documents(
    data_dir: str = "../data",
    chunk_size: int = 512,
    chunk_overlap: int = 20
) -> List[Document]:
    """
    加载文档并进行分块处理
    """
    # 1. 加载文档
    print(f"从 {data_dir} 加载文档...")
    reader = SimpleDirectoryReader(
        input_dir=data_dir,
        recursive=True,
        required_exts=[".pdf", ".docx", ".txt"],
    )
    
    documents = reader.load_data()
    print(f"成功加载 {len(documents)} 个文档")
    
    # 2. 添加元数据（扩展LLM课程的元数据管理）
    for doc in documents:
        file_name = doc.metadata.get("file_name", "unknown_source")
        doc.metadata["source"] = file_name
        doc.metadata["processed_date"] = str(datetime.now().date())
        doc.metadata["course"] = "RAG"  # 新增课程标识
        
    # 3. 文档分块（优化LLM课程的分块策略）
    print(f"对文档进行分块处理 (chunk_size={chunk_size}, chunk_overlap={chunk_overlap})")
    parser = SentenceSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separator="\n\n"  # 优先按段落分割
    )
    
    nodes = parser.get_nodes_from_documents(documents)
    
    print(f"文档分块完成，共生成 {len(nodes)} 个文档块")
    print(f"示例块内容: {nodes[0].text[:100]}...")
    
    return nodes

if __name__ == "__main__":
    nodes = load_and_process_documents()
    
    # 保存处理结果
    import pickle
    with open("processed_nodes.pkl", "wb") as f:
        pickle.dump(nodes, f)
    print("文档处理结果已保存至 processed_nodes.pkl")

In [ ]:
# Assuming you have run the load_and_process_documents function
nodes = load_and_process_documents(data_dir="../data")  # 指向文档的路径

# Initialize LocalRAG without business_cases
documents_local_rag = LocalRAG(business_cases=None)
# Load the processed documents
documents_local_rag.load_documents(nodes)

# Now you can query as usual
query = "哪些企业通过定价策略实现了市场渗透？请分析其核心逻辑。"
answer = documents_local_rag.query(query, top_k=2)
print(f"RAG回答：{answer}")

#### 2.2.3 高级性能优化（仅作了解）

#### 2.2.3.1 异步查询

```python
async def batch_query(self, queries: list) -> list:
    """批量执行异步查询（RAG性能优化技术）"""
    tasks = [self.async_query(q) for q in queries]
    results = await asyncio.gather(*tasks)
    return results
```

#### 2.2.3.2 索引优化

```python
index_params_list = [
    {"index_type": "IVF_FLAT", "params": {"nlist": 128}},  # 基础索引
    {"index_type": "HNSW", "params": {"M": 16, "efConstruction": 256}}  # 优化索引
]
```

## 2.3 使用Cherry Studio构建知识库

### 2.3.1 Cherry Studio知识库功能概述

Cherry Studio提供了完整的知识库管理功能，支持从文档导入、向量化处理到检索优化的全流程可视化操作。根据[官方文档](https://docs.cherry-ai.com/knowledge-base/knowledge-base)，其核心功能包括：

- **多源文档导入**：支持PDF、Word、TXT等多种格式
- **智能分块处理**：自动优化文档分块策略
- **向量存储管理**：支持Milvus/FAISS等多种向量数据库
- **检索策略配置**：灵活调整检索参数和相似度阈值
- **知识库测试**：内置查询测试和结果分析工具

拓展资源：
- **Cherry Studio官方文档**：https://docs.cherry-ai.com/knowledge-base/knowledge-base
- **知识库最佳实践**：https://docs.cherry-ai.com/knowledge-base/best-practices
- **高级检索策略**：https://docs.cherry-ai.com/knowledge-base/advanced-retrieval
- **性能优化指南**：https://docs.cherry-ai.com/knowledge-base/performance-tuning

#### 2.3.1.1 创建新知识库

1. 启动Cherry Studio，进入"知识库"模块
2. 点击"新建知识库"，填写基本信息：
   - 名称：企业知识库
   - 描述：用于存储和检索企业内部文档
   - 存储位置：本地
   - 向量数据库：Milvus（已在环境准备中配置）
3. 点击"创建"，系统将初始化知识库结构

#### 2.3.1.2 配置知识库参数

1. 进入知识库详情页，切换到"设置"标签
2. 配置文档处理参数：
   ```json
   {
     "chunk_size": 512,
     "chunk_overlap": 50,
     "separator": "\n\n",
     "include_metadata": true
   }
   ```
3. 配置嵌入模型：
   - 模型名称：BAAI/bge-base-en-v1.5
   - 批量大小：16
   - 设备：自动
4. 点击"保存配置"并应用

### 2.3.2 文档导入与管理

#### 2.3.2.1 导入文档
1. 进入"文档管理"标签，点击"导入文档"
2. 选择导入方式：
   - 方式1：本地上传（支持多文件批量上传）
   - 方式2：目录同步（监控指定目录自动导入）
3. 选择测试文档（可使用课程提供的sample_docs.zip）
4. 点击"开始导入"，系统将自动处理文档

#### 2.3.2.2 文档处理监控
1. 导入过程中，可在"任务中心"查看处理进度
2. 处理完成后，查看文档统计信息：
   - 总文档数：X个
   - 总文档块数：Y个
   - 平均分块大小：Z tokens
3. 检查是否有处理失败的文档，点击"重试"处理异常文档

### 2.3.3 检索策略配置

#### 2.3.3.1 基础检索配置
1. 进入"检索设置"标签，配置基础参数：
   - 相似度阈值：0.75
   - 返回结果数：5
   - 重排序：启用（基于BM25）
2. 点击"测试检索"，输入测试查询："企业核心产品有哪些？"
3. 查看检索结果和相关性评分，调整参数优化结果

#### 2.3.3.2 高级检索策略（混合检索配置）
1. 在"检索设置"中启用"混合检索"
2. 配置参数：
   - 密集向量权重：0.7
   - 稀疏向量权重：0.3
   - 交叉编码器重排序：启用
3. 点击"保存并应用"，系统将自动重建索引
4. 对比启用前后的检索效果差异

### 2.3.4 知识库应用开发


#### 2.3.4.1 创建检索应用
1. 进入"应用构建"模块，点击"新建应用"
2. 选择模板："知识库问答应用"
3. 配置应用参数：
   - 名称：企业知识库问答
   - 关联知识库：选择之前创建的"企业知识库"
   - 模型：deepseek-r1:7b等本地Ollama生成模型
   - 提示词模板：使用RAG专用模板
4. 点击"创建应用"，系统自动生成应用界面

#### 2.3.4.2 应用测试与优化
1. 进入应用详情页，点击"预览"
2. 在测试界面输入问题，测试不同类型查询：
   - 事实型："企业成立时间？"
   - 概念型："什么是核心竞争力？"
   - 分析型："分析产品市场优势"
3. 查看回答质量和引用来源，记录需要优化的案例
4. 进入"优化中心"，针对低质量回答调整：
   - 修改分块参数
   - 优化提示词模板
   - 调整检索策略

### 2.3.5 知识库自动更新

1. 进入知识库"设置"→"自动更新"
2. 配置更新策略：
   - 更新频率：每周日凌晨2点
   - 更新范围：新增文档和修改过的文档
   - 通知方式：邮件通知
3. 点击"启用自动更新"

### 2.3.6 常见问题与解决方案

#### 文档导入失败
**问题**：PDF文档导入后内容为空  
**解决方案**：
1. 检查PDF是否加密或扫描件（需OCR处理）
2. 进入"设置"→"文档处理"，启用"高级PDF解析"
3. 重新导入文档

#### 检索结果相关性低
**问题**：查询结果与问题相关性差  
**解决方案**：
1. 降低相似度阈值（如从0.8调整至0.7）
2. 增加chunk_overlap至100
3. 尝试不同的嵌入模型（如切换至bge-large模型）

#### 知识库体积过大
**问题**：知识库文档过多导致检索缓慢  
**解决方案**：
1. 启用"分层检索"（设置知识库层级结构）
2. 配置"自动归档"策略，将旧文档移至归档库
3. 优化索引参数，使用IVF_FLAT索引类型

## 2.4 课程作业：小型知识库系统开发

### 2.4.1 作业目标
使用Cherry Studio构建一个小型的知识库系统，实现文档管理、智能检索和问答功能。

### 2.4.2 具体要求
1. **知识库构建**：
   - 导入至少10篇不同类型的文档
   - 配置混合检索策略
   - 优化分块和嵌入参数

2. **应用开发**：
   - 创建知识库问答应用
   - 实现自定义提示词模板
   - 添加结果可视化功能

3. **性能优化**：
   - 对比不同检索策略的效果
   - 分析并优化低相关性查询案例
   - 生成性能测试报告

### 提交内容
- 知识库配置截图
- 应用界面截图
- 5组测试查询的问答记录
- 优化前后的性能对比报告
- 技术总结（500字以内）